In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Стъпка 1: Дефиниране на Pydantic модели за структурирани изходи

Тези модели дефинират **схемата**, която агентите ще връщат. Използването на `response_format` с Pydantic осигурява:
- ✅ Извличане на данни с гарантирана типова безопасност
- ✅ Автоматична валидация
- ✅ Без грешки при парсинг на свободен текст
- ✅ Лесно условно маршрутизиране въз основа на полета


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Стъпка 2: Създаване на инструмента за резервация на хотел

Този инструмент е това, което **availability_agent** ще използва, за да провери дали има налични стаи. Използваме декоратора `@ai_function`, за да:
- Превърнем Python функция в инструмент, който може да бъде извикан от AI
- Автоматично генерираме JSON схема за LLM
- Обработваме валидирането на параметрите
- Позволим автоматично извикване от агенти

За този демонстрационен пример:
- **Стокхолм, Сиатъл, Токио, Лондон, Амстердам** → Има налични стаи ✅
- **Всички други градове** → Няма налични стаи ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Стъпка 3: Дефиниране на функции за условия за маршрутизиране

Тези функции проверяват отговора на агента и определят кой път да се следва в работния процес.

**Основен модел:**
1. Проверете дали съобщението е `AgentExecutorResponse`
2. Анализирайте структурираните данни (Pydantic модел)
3. Върнете `True` или `False`, за да контролирате маршрутизирането

Работният процес ще оценява тези условия на **връзките**, за да реши кой изпълнител да извика следващ.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Стъпка 4: Създаване на персонализиран изпълнител за показване

Изпълнителите са компоненти на работния процес, които извършват трансформации или странични ефекти. Използваме декоратора `@executor`, за да създадем персонализиран изпълнител, който показва крайния резултат.

**Основни концепции:**
- `@executor(id="...")` - Регистрира функция като изпълнител на работния процес
- `WorkflowContext[Never, str]` - Типови подсказки за вход/изход
- `ctx.yield_output(...)` - Предава крайния резултат от работния процес


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Стъпка 5: Зареждане на променливи на средата

Конфигурирайте клиента на LLM. Този пример работи с:
- **GitHub модели** (Безплатен план с GitHub токен)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Стъпка 6: Създаване на AI агенти със структурирани изходи

Създаваме **три специализирани агента**, всеки обвит в `AgentExecutor`:

1. **availability_agent** - Проверява наличността на хотели с помощта на инструмента
2. **alternative_agent** - Предлага алтернативни градове (когато няма свободни стаи)
3. **booking_agent** - Насърчава резервация (когато има свободни стаи)

**Основни характеристики:**
- `tools=[hotel_booking]` - Осигурява инструмента на агента
- `response_format=PydanticModel` - Принуждава структурирания JSON изход
- `AgentExecutor(..., id="...")` - Обвива агента за използване в работния процес


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Стъпка 7: Създаване на работния процес с условни връзки

Сега използваме `WorkflowBuilder`, за да изградим графиката с условно маршрутизиране:

**Структура на работния процес:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Основни методи:**
- `.set_start_executor(...)` - Задава началната точка
- `.add_edge(from, to, condition=...)` - Добавя условна връзка
- `.build()` - Финализира работния процес


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Стъпка 8: Изпълнение на тестов случай 1 - Град БЕЗ наличност (Париж)

Нека тестваме пътя **без наличност**, като поискаме хотели в Париж (който няма стаи в нашата симулация).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Стъпка 9: Изпълнение на тестов случай 2 - Град С Наличност (Стокхолм)

Сега нека тестваме пътя на **наличност**, като заявим хотели в Стокхолм (който има свободни стаи в нашата симулация).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Основни изводи и следващи стъпки

### ✅ Какво научихте:

1. **Модел WorkflowBuilder**
   - Използвайте `.set_start_executor()`, за да зададете начална точка
   - Използвайте `.add_edge(from, to, condition=...)` за условно маршрутизиране
   - Извикайте `.build()`, за да финализирате работния процес

2. **Условно маршрутизиране**
   - Функциите за условия проверяват `AgentExecutorResponse`
   - Анализирайте структурирани изходи, за да вземете решения за маршрутизиране
   - Върнете `True`, за да активирате връзка, `False`, за да я пропуснете

3. **Интеграция на инструменти**
   - Използвайте `@ai_function`, за да преобразувате Python функции в AI инструменти
   - Агенти автоматично извикват инструменти, когато е необходимо
   - Инструментите връщат JSON, който агентите могат да анализират

4. **Структурирани изходи**
   - Използвайте Pydantic модели за безопасно извличане на данни по тип
   - Задайте `response_format=MyModel`, когато създавате агенти
   - Анализирайте отговори с `Model.model_validate_json()`

5. **Персонализирани изпълнители**
   - Използвайте `@executor(id="...")`, за да създадете компоненти на работния процес
   - Изпълнителите могат да трансформират данни или да извършват странични ефекти
   - Използвайте `ctx.yield_output()`, за да генерирате резултати от работния процес

### 🚀 Приложения в реалния свят:

- **Резервации за пътувания**: Проверка на наличност, предлагане на алтернативи, сравнение на опции
- **Обслужване на клиенти**: Маршрутизиране според типа проблем, настроение, приоритет
- **Електронна търговия**: Проверка на наличност, предлагане на алтернативи, обработка на поръчки
- **Модерация на съдържание**: Маршрутизиране според токсичност, потребителски сигнали
- **Одобрителни процеси**: Маршрутизиране според сума, роля на потребителя, ниво на риск
- **Многоетапна обработка**: Маршрутизиране според качество на данни, пълнота

### 📚 Следващи стъпки:

- Добавете по-сложни условия (множество критерии)
- Реализирайте цикли с управление на състоянието на работния процес
- Добавете подработни процеси за многократна употреба
- Интегрирайте с реални API (резервации на хотели, системи за наличност)
- Добавете обработка на грешки и резервни пътища
- Визуализирайте работните процеси с вградените инструменти за визуализация



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
